In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path


DATA_DIR = Path("data")  
chemin1 = DATA_DIR / "source_system_a.xlsx"
chemin2 = DATA_DIR / "source_system_b.xlsx"

df  = pd.read_excel(chemin1)
df2 = pd.read_excel(chemin2)

# Colonnes nécessaires (Sans nom d'entreprise)
colonnes_utiles = [
    "Lib Famille",
    "Lib Sous-Famille",
    "Lib Gamme",
    "Nom du fournisseur",
    "Article",
    "Désignation article",
    "Unité UF Four",
    "Montant Net-net",
    "Montant Brut",
    "Poids net",
    "UF Organisation",           
    "Libellé du conditionnement",
]


renames_eventuels = {
    "UF x": "UF Organisation"   
}
df  = df.rename(columns=renames_eventuels)
df2 = df2.rename(columns=renames_eventuels)

# Sélectionne les colonnes
df  = df[[c for c in colonnes_utiles if c in df.columns]].copy()
df2 = df2[[c for c in colonnes_utiles if c in df2.columns]].copy()

df["Fournisseur"]  = "SYSTEM_A"
df2["Fournisseur"] = "SYSTEM_B"

df_final = pd.concat([df, df2], ignore_index=True)

df_final["QuantiteLibelle"] = (
    df_final["Désignation article"].astype(str).str.extract(r"(\d+\.?\d*)")[0].astype(float)
)
df_final["libelle"] = (
    df_final["Désignation article"].astype(str).str.replace(r"\d+", "", regex=True).str.strip()
)

#3NET FRS

DISCOUNTS = {
    "SUPPLIER_A": 0.90,   
    "SUPPLIER_B": 0.86,   
}

df_final["fournisseur_normalise"] = (
    df_final["Nom du fournisseur"]
        .replace({
            "x": "SUPPLIER_A",
            "y": "SUPPLIER_B"
        })
        .fillna("OTHER")
)

df_final["3netmontant"] = (
    df_final.apply(
        lambda r: DISCOUNTS.get(r["fournisseur_normalise"], 1.0) * r["Montant Net-net"],
        axis=1
    )
)

#  Conditionnement
df_final["Quantite_conditionnement"] = (
    df_final["Libellé du conditionnement"].astype(str).str.extract(r"(\d+\.?\d*)")[0].astype(float)
)
df_final["Type_de_conditionnement"] = (
    df_final["Libellé du conditionnement"].astype(str).str.replace(r"\d+", "", regex=True).str.strip()
)


df_final["prixunite3net"] = np.where(
    df_final["Quantite_conditionnement"].fillna(0) > 0,
    df_final["3netmontant"] / df_final["Quantite_conditionnement"],
    np.nan
)

# Export 
OUTPUT_DIR = Path("outputs")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
df_final.to_excel(OUTPUT_DIR / "comparison_results.xlsx", index=False)
